In [2]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda')

In [3]:
path = './'

In [4]:
train_png = sorted(glob(path + 'data/train/*.png'))
test_png = sorted(glob(path + 'data/test/*.png'))

In [5]:
len(train_png), len(test_png)

(4277, 2154)

In [8]:
train_y = pd.read_csv(path +"data/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [9]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (384, 384),interpolation = cv2.INTER_AREA)
    return img

In [10]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|███████████████████████████████████████| 2154/2154 [01:02<00:00, 34.61it/s]


In [11]:
np.save(path + 'data/train_imgs_384', np.array(train_imgs))
np.save(path + 'data/test_imgs_384', np.array(test_imgs))

In [13]:
train_imgs = np.load(path + 'data/train_imgs_384.npy')
test_imgs = np.load(path + 'data/test_imgs_384.npy')

In [14]:
meanRGB = [np.mean(x, axis=(0,1)) for x in train_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in train_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("train 평균",meanR, meanG, meanB)
print("train 표준편차",stdR, stdG, stdB)

train 평균 0.4330380901867049 0.4034575319032911 0.39415050509784405
train 표준편차 0.1815717110252788 0.17403455556798705 0.16323395055036488


In [15]:
meanRGB = [np.mean(x, axis=(0,1)) for x in test_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in test_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("test 평균",meanR, meanG, meanB)
print("test 표준편차",stdR, stdG, stdB)

test 평균 0.41825619520929724 0.3931011906330291 0.386631764639131
test 표준편차 0.19505524270747931 0.19005280951759498 0.18053225852732663


In [16]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
          train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.433038, 0.403458, 0.394151],
                                     std = [0.181572, 0.174035, 0.163234]),
                transforms.RandomAffine((-45, 45)),
                
            ])
          img = train_transform(img)
        if self.mode == 'test':
          test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.418256, 0.393101, 0.386632],
                                     std = [0.195055, 0.190053, 0.185323])
            ])
          img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=88, drop_path_rate = 0.2)
        if self.mode == 'test':
          self.model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=88, drop_path_rate = 0)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [17]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [18]:
def main(seed = 2022):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    
main(2022)

In [19]:
import gc

cv = StratifiedKFold(n_splits = 5, random_state = 2022,shuffle=True)
batch_size = 32
epochs = 70
pred_ensemble = []


for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):
  print("----------fold_{} start!----------".format(idx))
  t_imgs, val_imgs = train_imgs[train_idx],  train_imgs[val_idx]
  t_labels, val_labels = np.array(train_labels)[train_idx], np.array(train_labels)[val_idx]

  # Train
  train_dataset = Custom_dataset(np.array(t_imgs), np.array(t_labels), mode='train')
  train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

  # Val
  val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), mode='test')
  val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

  gc.collect()
  torch.cuda.empty_cache()
  best=0

  model = Network().to(device)

  optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay = 2e-2)
  criterion = nn.CrossEntropyLoss()
  scaler = torch.cuda.amp.GradScaler()  

  best_f1 = 0
  early_stopping = 0
  for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    train_f1 = score_function(train_y, train_pred)
    state_dict= model.state_dict()
    model.eval()
    with torch.no_grad():
      val_loss = 0 
      val_pred = []
      val_y = []
      

      for batch in (val_loader):
        x_val = torch.tensor(batch[0], dtype = torch.float32, device = device)
        y_val = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred_val = model(x_val)
        loss_val = criterion(pred_val, y_val)

        val_loss += loss_val.item()/len(val_loader)
        val_pred += pred_val.argmax(1).detach().cpu().numpy().tolist()
        val_y += y_val.detach().cpu().numpy().tolist()
      val_f1 = score_function(val_y, val_pred)

      if val_f1 > best_f1:
        best_epoch = epoch
        best_loss = val_loss
        best_f1 = val_f1
        early_stopping = 0

        torch.save({'epoch':epoch,
                    'state_dict':state_dict,
                    'optimizer': optimizer.state_dict(),
                    'scaler': scaler.state_dict(),
             }, path +'best_model_{}.pth'.format(idx))
        print('-----------------SAVE:{} epoch----------------'.format(best_epoch+1))
      else:
          early_stopping += 1

            # Early Stopping
      if early_stopping == 20:
        TIME = time.time() - start
        print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
        print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
        print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')
        break

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')

----------fold_0 start!----------


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b3_ra2-cf984f9c.pth" to /home/lab20/.cache/torch/hub/checkpoints/efficientnet_b3_ra2-cf984f9c.pth


-----------------SAVE:1 epoch----------------
epoch : 1/70    time : 96s/6595s
TRAIN    loss : 1.18217    f1 : 0.18602
Val    loss : 0.62408    f1 : 0.25515
-----------------SAVE:2 epoch----------------
epoch : 2/70    time : 88s/5969s
TRAIN    loss : 0.55791    f1 : 0.36260
Val    loss : 0.46246    f1 : 0.40098
-----------------SAVE:3 epoch----------------
epoch : 3/70    time : 89s/5965s
TRAIN    loss : 0.39060    f1 : 0.51017
Val    loss : 0.33407    f1 : 0.54776
-----------------SAVE:4 epoch----------------
epoch : 4/70    time : 89s/5895s
TRAIN    loss : 0.28324    f1 : 0.63564
Val    loss : 0.35030    f1 : 0.60441
-----------------SAVE:5 epoch----------------
epoch : 5/70    time : 90s/5851s
TRAIN    loss : 0.24072    f1 : 0.68668
Val    loss : 0.22283    f1 : 0.65025
-----------------SAVE:6 epoch----------------
epoch : 6/70    time : 90s/5765s
TRAIN    loss : 0.16152    f1 : 0.80172
Val    loss : 0.19800    f1 : 0.70202
epoch : 7/70    time : 90s/5679s
TRAIN    loss : 0.12204  

epoch : 17/70    time : 88s/4682s
TRAIN    loss : 0.03270    f1 : 0.97270
Val    loss : 0.18340    f1 : 0.79968
-----------------SAVE:18 epoch----------------
epoch : 18/70    time : 89s/4620s
TRAIN    loss : 0.03395    f1 : 0.97581
Val    loss : 0.17472    f1 : 0.81904
-----------------SAVE:19 epoch----------------
epoch : 19/70    time : 89s/4526s
TRAIN    loss : 0.02126    f1 : 0.97604
Val    loss : 0.15110    f1 : 0.81920
-----------------SAVE:20 epoch----------------
epoch : 20/70    time : 89s/4433s
TRAIN    loss : 0.01831    f1 : 0.97892
Val    loss : 0.12764    f1 : 0.83933
epoch : 21/70    time : 88s/4332s
TRAIN    loss : 0.03247    f1 : 0.96159
Val    loss : 0.19551    f1 : 0.76485
epoch : 22/70    time : 88s/4229s
TRAIN    loss : 0.04353    f1 : 0.95430
Val    loss : 0.19120    f1 : 0.78784
epoch : 23/70    time : 88s/4155s
TRAIN    loss : 0.02937    f1 : 0.96254
Val    loss : 0.18549    f1 : 0.78082
-----------------SAVE:24 epoch----------------
epoch : 24/70    time : 89s/

-----------------SAVE:13 epoch----------------
epoch : 13/70    time : 89s/5062s
TRAIN    loss : 0.03150    f1 : 0.96227
Val    loss : 0.11262    f1 : 0.81828
-----------------SAVE:14 epoch----------------
epoch : 14/70    time : 89s/4971s
TRAIN    loss : 0.03996    f1 : 0.96178
Val    loss : 0.12356    f1 : 0.83326
epoch : 15/70    time : 88s/4854s
TRAIN    loss : 0.02774    f1 : 0.97049
Val    loss : 0.12104    f1 : 0.80379
-----------------SAVE:16 epoch----------------
epoch : 16/70    time : 89s/4795s
TRAIN    loss : 0.02934    f1 : 0.96730
Val    loss : 0.10654    f1 : 0.86568
epoch : 17/70    time : 88s/4678s
TRAIN    loss : 0.03577    f1 : 0.96355
Val    loss : 0.10525    f1 : 0.83270
epoch : 18/70    time : 88s/4579s
TRAIN    loss : 0.03424    f1 : 0.95866
Val    loss : 0.15297    f1 : 0.84158
epoch : 19/70    time : 88s/4506s
TRAIN    loss : 0.03525    f1 : 0.95924
Val    loss : 0.14184    f1 : 0.81713
epoch : 20/70    time : 88s/4413s
TRAIN    loss : 0.03535    f1 : 0.95188
V

epoch : 43/70    time : 88s/2371s
TRAIN    loss : 0.00376    f1 : 0.99704
Val    loss : 0.17664    f1 : 0.79670
epoch : 44/70    time : 88s/2279s
TRAIN    loss : 0.00359    f1 : 0.99582
Val    loss : 0.16457    f1 : 0.79994
epoch : 45/70    time : 88s/2192s
TRAIN    loss : 0.00872    f1 : 0.99014
Val    loss : 0.14488    f1 : 0.81839
epoch : 46/70    time : 88s/2105s
TRAIN    loss : 0.02030    f1 : 0.97969
Val    loss : 0.14038    f1 : 0.81386
epoch : 47/70    time : 88s/2019s
TRAIN    loss : 0.02177    f1 : 0.97960
Val    loss : 0.16513    f1 : 0.83227
epoch : 48/70    time : 88s/1932s
TRAIN    loss : 0.02002    f1 : 0.98463
Val    loss : 0.17121    f1 : 0.78402
epoch : 49/70    time : 88s/1846s
TRAIN    loss : 0.01662    f1 : 0.98332
Val    loss : 0.14186    f1 : 0.82319
epoch : 50/70    time : 88s/1756s
TRAIN    loss : 0.01315    f1 : 0.98577
Val    loss : 0.16602    f1 : 0.82177
----------fold_4 start!----------
-----------------SAVE:1 epoch----------------
epoch : 1/70    time : 8

epoch : 61/70    time : 89s/799s
TRAIN    loss : 0.01115    f1 : 0.99198
Val    loss : 0.21695    f1 : 0.79115
epoch : 62/70    time : 89s/712s
TRAIN    loss : 0.01344    f1 : 0.99295
Val    loss : 0.21281    f1 : 0.78018
epoch : 63/70    time : 89s/623s
TRAIN    loss : 0.01466    f1 : 0.98628
Val    loss : 0.31865    f1 : 0.75117
epoch : 64/70    time : 89s/535s
TRAIN    loss : 0.01711    f1 : 0.98333
Val    loss : 0.14645    f1 : 0.82374
epoch : 65/70    time : 89s/447s
TRAIN    loss : 0.01841    f1 : 0.98278
Val    loss : 0.16727    f1 : 0.76778


In [20]:
pred_ensemble = []
batch_size = 32
# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

for i in range(5):
  model_test = Network(mode = 'test').to(device)
  model_test.load_state_dict(torch.load((path+'best_model_{}.pth'.format(i)))['state_dict'])
  model_test.eval()
  pred_prob = []
  with torch.no_grad():
      for batch in (test_loader):
          x = torch.tensor(batch[0], dtype = torch.float32, device = device)
          with torch.cuda.amp.autocast():
              pred = model_test(x)
              pred_prob.extend(pred.detach().cpu().numpy())
      pred_ensemble.append(pred_prob)

In [21]:
len(pred_ensemble)

5

In [22]:
pred = (np.array(pred_ensemble[0])+ np.array(pred_ensemble[1])+ np.array(pred_ensemble[3]) + np.array(pred_ensemble[4]) )/4
f_pred = np.array(pred).argmax(1).tolist()

In [23]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [24]:
submission = pd.read_csv(path + "data/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [25]:
submission.to_csv(path + "data/submission/b3_norm_epoch70_4_2.csv", index = False)

### 모델 학습

사전 학습 모델의 성능 파악을 할 때 Fold 학습은 실행 시간이 오래걸려서 fold를 나누지 않은 데이터에 대해서 학습을 진행하고 성능을 비교하였습니다.

In [26]:
batch_size = 32
epochs = 30

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [27]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [28]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay = 1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 

batch_size = 32
epochs = 30

best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

epoch : 1/30    time : 104s/3005s
TRAIN    loss : 1.63582    f1 : 0.14407
epoch : 2/30    time : 103s/2874s
TRAIN    loss : 0.79794    f1 : 0.18042
epoch : 3/30    time : 104s/2808s
TRAIN    loss : 0.60019    f1 : 0.27673
epoch : 4/30    time : 105s/2723s
TRAIN    loss : 0.49201    f1 : 0.36683
epoch : 5/30    time : 105s/2630s
TRAIN    loss : 0.40832    f1 : 0.46088
epoch : 6/30    time : 105s/2529s
TRAIN    loss : 0.34700    f1 : 0.52105
epoch : 7/30    time : 105s/2424s
TRAIN    loss : 0.30297    f1 : 0.58867
epoch : 8/30    time : 106s/2321s
TRAIN    loss : 0.25802    f1 : 0.63940
epoch : 9/30    time : 106s/2217s
TRAIN    loss : 0.21819    f1 : 0.72477
epoch : 10/30    time : 105s/2110s
TRAIN    loss : 0.18276    f1 : 0.76222
epoch : 11/30    time : 106s/2005s
TRAIN    loss : 0.16197    f1 : 0.78331
epoch : 12/30    time : 106s/1900s
TRAIN    loss : 0.13585    f1 : 0.84183
epoch : 13/30    time : 106s/1795s
TRAIN    loss : 0.13361    f1 : 0.83834
epoch : 14/30    time : 106s/1689s

### 추론

In [29]:
model.eval()
f_pred = []
pred_prob = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
            pred_prob.extend(pred.detach().cpu().numpy())
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [30]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

### 제출물 생성

In [31]:
submission = pd.read_csv(path + "data/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [33]:
submission.to_csv(path + "data/submission/b3_norm.csv", index = False)